# Import Library

In [1]:
import pickle
import pandas as pd, numpy as np

# from elang.plot.utils import plot2d, plotNeighbours
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from scipy import spatial

from tqdm import tqdm
import helper
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
import importlib
importlib.reload(helper)

<module 'helper' from 'C:\\Users\\Meta Mahyarani\\Perangkat_AJS\\Tugas Kelompok\\Tugas kedua\\helper.py'>

# Simpan kedalam file tweet yang disimpan kedalam data

In [3]:
with open('data/tweet', 'rb') as file:
    tweet = pickle.load(file)

In [4]:
x = tweet[1]

In [5]:
x.entities

{'hashtags': [{'text': 'JagaUlama', 'indices': [129, 139]},
  {'text': 'cabutomnibuslaw', 'indices': [141, 157]},
  {'text': 'ganyangpki', 'indices': [159, 170]},
  {'text': 'opposite6890', 'indices': [172, 185]},
  {'text': 'resistence6890', 'indices': [187, 202]},
  {'text': 'cyber_303', 'indices': [204, 214]},
  {'text': 'd34th_5kull', 'indices': [216, 228]}],
 'symbols': [],
 'user_mentions': [],
 'urls': []}

# Read Data

In [6]:
# Read pre-processed text
tweets = pd.read_csv('tweet_omnibuslaw.csv')

# Lakukan Inspect Data Berdasarkan 5 data teratas

In [7]:
tweets.head()
tweets

,text,id,userid,username,created_at,likes,retweets,isquote,is_retweet,mentions,hashtags,reply_to,geo,place,text_clean
0,RT @QaillaAsyiqah: Update ~ Di depan Gedung Ma...,1335948302521516037,1006725144398610437,athrun_cloud,2020-12-07 14:04:30,0,28,False,True,['QaillaAsyiqah'],[],NaN,NaN,NaN,rt @qaillaasyiqah: update ~ mahkamah konstitus...
1,"7. Bila situasi memungkinkan, maka ganggu seca...",1335944819944800256,1127166517244612608,DoeltzRahman,2020-12-07 13:50:40,0,0,False,False,[],"['JagaUlama', 'resistance6890', 'mositidakperc...",1.335945e+18,NaN,NaN,"7. situasi , ganggu fisik serius dwi waryadi, ..."
2,RT @QaillaAsyiqah: Gugatan UU 11 TAHUN 2020 Te...,1335943132479848449,3192210907,celoteh_makdin,2020-12-07 13:43:58,0,18,False,True,['QaillaAsyiqah'],['CabutOmnibuslaw'],NaN,NaN,NaN,rt @qaillaasyiqah: gugatan uu 11 2020 cipta mk...
3,RT @QaillaAsyiqah: Gugatan UU 11 TAHUN 2020 Te...,1335942059849465858,109373771,sang_alim,2020-12-07 13:39:42,0,18,False,True,['QaillaAsyiqah'],['CabutOmnibuslaw'],NaN,NaN,NaN,rt @qaillaasyiqah: gugatan uu 11 2020 cipta mk...
4,RT @QaillaAsyiqah: Gugatan UU 11 TAHUN 2020 Te...,1335940219808333824,1186924460588191744,Comrhang_Romeh,2020-12-07 13:32:23,0,18,False,True,['QaillaAsyiqah'],['CabutOmnibuslaw'],NaN,NaN,NaN,rt @qaillaasyiqah: gugatan uu 11 2020 cipta mk...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,@zarazettirazr Lanjutkan...umur panjang perjua...,1332622788465627136,483905465,mulyono379,2020-11-28 09:50:06,0,0,False,False,['zarazettirazr'],"['CabutOmnibuslaw', 'MosiTidakPercaya', 'Kamib...",1.332595e+18,NaN,"Place(id='2cd5efeb7102867f', url='https://api....",@zarazettirazr lanjutkan...umur perjuangan . #...
246,Tarikkkk Mang Dudung dulu braayy....💃💃\n\n#Kam...,1332616749150212096,1131775739278180352,HeartOds,2020-11-28 09:26:06,4,0,False,False,[],"['KamibersamaIBHRS', 'JeratJeratPolitik', 'Ani...",NaN,NaN,NaN,tarikkkk mang dudung braayy.... #kamibersamaib...
247,"@CybSqu buat apa ada wakil,ada sekda,!!!! mema...",1332609896114651136,1314092842835308544,zulhamkenzo,2020-11-28 08:58:52,0,0,False,False,['CybSqu'],['CabutOmnibuslaw'],1.332306e+18,NaN,NaN,"@cybsqu wakil, sekda,!!!! memangnya aturan !!!..."
248,#KamibersamaIBHRS \n#KamiBersamaTNIdanFPI \nJG...,1332593848694894593,542565359,mathsoul1,2020-11-28 07:55:06,0,0,True,False,[],"['KamibersamaIBHRS', 'KamiBersamaTNIdanFPI', '...",NaN,NaN,NaN,#kamibersamaibhrs #kamibersamatnidanfpi jgn ob...


# Persiapkan data kedalam username, is_retweet, mentions dan type

In [8]:
# edges = df[['username', 'mentions', 'type']]
# edges.columns = ['from', 'to','type']
# edges['count'] = 1
# edges = edges.groupby(['from', 'to', 'type']).sum().reset_index()

In [9]:
# edges

In [10]:
edges = pd.read_csv("edges_baru.csv")
edges

,from,to,type,count
0,10_igun,Muslim_AntiPKI,retweet,1
1,1MenujuDamai,QaillaAsyiqah,retweet,1
2,2ikhtiar,QaillaAsyiqah,retweet,1
3,30DetikTakHenti,QaillaAsyiqah,retweet,1
4,AJazuli5,JameelaNajma,retweet,1
...,...,...,...,...
209,ttaufik124,SaveMoslem1,retweet,1
210,ttaufik124,mathsoul1,retweet,1
211,ucokseparoh,QaillaAsyiqah,retweet,2
212,wahdinata,kanopimedia,retweet,1


In [11]:
condition = edges['type'] == "retweet"
edges_retweet = edges[condition]
edges_retweet = edges_retweet.drop(columns=['type', 'count'], index= 0)
edges_retweet

,from,to
1,1MenujuDamai,QaillaAsyiqah
2,2ikhtiar,QaillaAsyiqah
3,30DetikTakHenti,QaillaAsyiqah
4,AJazuli5,JameelaNajma
5,Abe_TSFE,kanopimedia
...,...,...
209,ttaufik124,SaveMoslem1
210,ttaufik124,mathsoul1
211,ucokseparoh,QaillaAsyiqah
212,wahdinata,kanopimedia


In [33]:
condition = edges['type'] == "mentions"
edges_mentions = edges[condition]
edges_mentions

edges_mentions = edges_mentions.drop(columns=['type', 'count'])
edges_mentions

,from,to
20,CasperBeWin1,AmnestyID
21,CasperBeWin1,amnesty
43,JokoPur36178526,Jurnal_KSN
49,Lini_ZQ,FsbnK
50,Lini_ZQ,KasbiIndo
51,Lini_ZQ,watchdoc_ID
60,Onyong_Marko,GreenpeaceID
63,RMl_id,jokowi
64,RMl_id,perkumpulanhuma
75,SemprulBae234,AmnestyID


In [34]:
edges_retweet.to_csv('edges_retweet_1.csv', index=False, header=False)
edges_mentions.to_csv('edges_mentions.csv', index=False, header=False)

In [13]:
# edges_baru = edges.drop(columns=['type', 'to'], index= 0)
# edges_baru

In [14]:
edges_baru_1 = edges.drop(columns=['type', 'count'])

In [15]:
edge_baru = pd.DataFrame(edges_baru_1).values
edge_baru

array([['10_igun', 'Muslim_AntiPKI'],
       ['1MenujuDamai', 'QaillaAsyiqah'],
       ['2ikhtiar', 'QaillaAsyiqah'],
       ['30DetikTakHenti', 'QaillaAsyiqah'],
       ['AJazuli5', 'JameelaNajma'],
       ['Abe_TSFE', 'kanopimedia'],
       ['Adhek_Byt', 'Muslim_AntiPKI'],
       ['Adhek_Byt', 'QaillaAsyiqah'],
       ['Alief1Ba', 'kanopimedia'],
       ['AlifKamal__', 'SeknasKPA'],
       ['AmiArdiwinata', 'kanopimedia'],
       ['AngrySipelebegu', 'SeknasKPA'],
       ['Aquafarming_ID', 'kanopimedia'],
       ['ArmanHlp', 'QaillaAsyiqah'],
       ['AsepRus04734801', 'QaillaAsyiqah'],
       ['BIfoenk', 'QaillaAsyiqah'],
       ['Bedjo57582', 'QaillaAsyiqah'],
       ['BinarisSitumor2', 'SeknasKPA'],
       ['BoneSangkala', 'Muslim_AntiPKI'],
       ['CarikeadilanDi', 'QaillaAsyiqah'],
       ['CasperBeWin1', 'AmnestyID'],
       ['CasperBeWin1', 'amnesty'],
       ['CiUniang4', 'Muslim_AntiPKI'],
       ['Comrhang_Romeh', 'QaillaAsyiqah'],
       ['DSXL3', 'QaillaAsyiqah'],
       

# Persiapkan data berdasarkan username

In [16]:
nodes = pd.DataFrame(set(edges[['from', 'to']].values.flatten()), columns=['username'])

In [17]:
nodes[0:11]

,username
0,2ikhtiar
1,IyanSatria10
2,reylbngklng_
3,satrisarkas
4,pemain_pinjaman
5,NajwaShihab
6,AlifKamal__
7,iwanalidarmawan
8,Abe_TSFE
9,DidiSuh61460608


In [38]:
# nodes_baru = nodes['username'].values

In [37]:
# G = nx.Graph()

In [36]:
# G.add_nodes_from(nodes_baru)
# G.add_edges_from(edge_baru)

In [35]:
# import community as community_louvain #pip install python louvain
# import matplotlib.pyplot as plt
# import matplotlib.cm as cm# define the graph

# partition = community_louvain.best_partition(G)
# partition

# Simpan Nodes Tersebut Kedalam File .csv

In [23]:
nodes.to_csv('nodes.csv', index=False)
# edges.to_csv('edges_baru.csv.csv')

# Menemukan Finding Key Player atau Influencer

**Finding Key Player**

In [24]:
topuser = tweets.copy()
topuser['total'] = topuser['likes'] + topuser['retweets']
topuser = topuser.groupby('username').sum().sort_values(by='total', ascending=False)[['likes', 'retweets']]
topuser.head()

,likes,retweets
username,,
borneoetnict,0.0,7382.0
ilyasrf63,0.0,1202.0
Herman84617774,0.0,1062.0
QaillaAsyiqah,239.0,65.0
ioppiss,0.0,281.0


# Melihat Tweet Berdasarkan Key Player

In [25]:
for i, row in  tweets[tweets['username'].isin(['borneoetnict', 'ilyasrf63'])].iterrows():
    print(f"Tweet from : {row['username']}")
    print(row['text'])
    print('____')

Tweet from : ilyasrf63
RT @Lini_ZQ: Masoh dari Bandung 🔊🔊🔊

#BatalkanOmnibusLaw #CabutOmnibuslaw #MosiTidakPercayaJokowi #MOSITIDAKDIPERCAYA https://t.co/k0rz6HYX…
____
Tweet from : borneoetnict
RT @KontraS: Laporkan Aparat di https://t.co/b5jP5vyI2W

#MosiTidakPercaya 
#CabutOmnibusLaw https://t.co/aB3z55shKH
____
Tweet from : borneoetnict
RT @Lini_ZQ: Begini laku aparat terhadap demonstran di Jogja 

#AparatJanganBrutal 
#CabutOmnibuslaw https://t.co/7Fq7Okn2Jm
____
